# Dealing with missing values

**Workflow**
    1. Count missing values
    2. Check why are they missing - missing values can be due to a lot of things
        - It really depends on the origin of the data and the context it was generated. For example, for a survey, a Salary field with an empty value, or a number 0, or an invalid value (a string for example) can be considered "missing data".
    3. Decide what to do with missing values based on the 2 step

**Its important to look at the data and understand why the data is missing**:
- Is this value missing because it wasn't recorded or because it doesn't exist?

If the value is missing because **it doesn't exist** (like the height of the oldest child of someone who doesn't have any children) then it doesn't make sense to try and guess what it migth be.

We can also **drop** if the missing values in a column rarely happend and occur at random, or if most of the column's values are missing



*On the other hand*, if a value is missing because it wasn't recorded, then we can try to guess what it might have been based on the other values in that column and row => **Imputation**


###### Strategies:
- Delete missing rows 
- Delete features that contain more than X% of missing values
- Replace with the next value
- Replace with mean (for numerical)
- Replace with mode (for categorical)
- Replace with median (for numerical)
- Impute with KNN Imputer
- Impute with Iterative Imputer

In [1]:
print('hello')

hello


In [2]:
import pandas as pd
import numpy as np

df = pd.read_csv('Building_Permits.csv')
df.shape

/usr/lib/python3/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (22,32) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


(198900, 43)

In [3]:
missing_values_count = df.isnull().sum()
missing_values_count

Permit Number                                  0
Permit Type                                    0
Permit Type Definition                         0
Permit Creation Date                           0
Block                                          0
Lot                                            0
Street Number                                  0
Street Number Suffix                      196684
Street Name                                    0
Street Suffix                               2768
Unit                                      169421
Unit Suffix                               196939
Description                                  290
Current Status                                 0
Current Status Date                            0
Filed Date                                     0
Issued Date                                14940
Completed Date                            101709
First Construction Document Date           14946
Structural Notification                   191978
Number of Existing S

In [4]:
df.shape

(198900, 43)

In [13]:
'''
    Percentage of cells that contain missing values!
'''
# how many total missing values do we have?
total_cells = np.product(df.shape)
total_missing = missing_values_count.sum()

# percent of data that is missing
percent_missing = (total_missing/total_cells) * 100
print(percent_missing ,'%')

26.26002315058403 %


##### Delete missing rows

By removing a whole observation just because that observation contains a missing value in one cell, we can loose relevant information that is in the other features/cells

*This technique is good when a feature has few missing values*

In [6]:
df_missing_rows_deleted = df.dropna()
df_missing_rows_deleted.shape

# In this case it deletes all the rows because all contain at least one missing value!!

(0, 43)

#####  Delete features that contain more than X% of missing values

This technique is good for features that have a high percentage of missing values.
If a certain features has a high percentage of missing values, it's not a good idea to try and gess the values because we don't have enough data to infer.

**First we have to check if the missing values in a features have a certain meaning**

After we still have to decide what to do with the other features that contain missing values but the percentage is lower than X%

In [22]:
total_rows = df.shape[0]
missing_feature_percentage = missing_values_count /total_rows*100
missing_feature_percentage

Permit Number                              0.000000
Permit Type                                0.000000
Permit Type Definition                     0.000000
Permit Creation Date                       0.000000
Block                                      0.000000
Lot                                        0.000000
Street Number                              0.000000
Street Number Suffix                      98.885872
Street Name                                0.000000
Street Suffix                              1.391654
Unit                                      85.178984
Unit Suffix                               99.014077
Description                                0.145802
Current Status                             0.000000
Current Status Date                        0.000000
Filed Date                                 0.000000
Issued Date                                7.511312
Completed Date                            51.135747
First Construction Document Date           7.514329
Structural N

In [30]:
# Drop features that have more than 30%
to_drop = missing_feature_percentage[missing_feature_percentage > 30]

In [27]:
df_missing_features_drop  =df.drop(to_drop.index.tolist(), 1)
df_missing_features_drop.shape

(198900, 34)

##### Replace with next value

This technique is great for time series data where the data is collected in short intervals of time. In these cases, the next value can be very similar to the missing value because the data has been collect almost at the same time

In [32]:
# bfill uses the next valid observation to fill gap.
# ffill uses the last valid observation
df_next_value = df.fillna(method='bfill')
df_next_value = df_next_value.fillna(method='ffill') # for the last row.

df_next_value.isnull().any().any()

False

##### Replace with mean (for numerical)

This technique has the advantage that is fast, but it only takes into account a single attribute thus not counting with the possible relations with other features (**Univariate Imputing)**. It also does not add new relevant information to the feature and can reduce the variability of the data

**Mean is most useful when the original data is not skewed**

In [ ]:
df.mean()

In [ ]:
df_with_mean = df.fillna(df.mean())
df_with_mean.isnull().any().any()

##### Replace with mode

The pros and cons of this technique are similar to the ones of replacing with mean

It works well with categorical features but can introduce bias in the data!

In [ ]:
df_with_mode = df.fillna(df.mean())
df_with_mode.isnull().any().any()

##### Replace with median

The pros and cons of this technique are similar to the ones of replacing with mean

But imputing with the median makes more sense than with the mean when the data is skewed

In [ ]:
df_with_median = df.fillna(df.median())
df_with_median.isnull().any().any()

#### Imputers that take into account the other features

The next two techiques are **very good** because they use the entire dataset to estimate the missing values. This is good because they don't work on an attribute level. They use the relationships between the different features in order to estimate the missing values

##### Impute with KNN Imputer

- Provides imputation for filling in missing values unsing the k-nearest neighbors approach
- By default, a euclidean distance metric that supports missing values is used to find the nearest neighbors
- Each missing feature is imputed using values from n_neighbors nearest neighbors that have a value for the feature.


In [ ]:
from sklearn.impute.KNNImputer

imputer = KNNImputer(n_neighbors = 10)
imputed_df = df.fit_transform(df.values)
imputed_df.isnull().any().any()


##### Impute with Iterative Imputer

- Models each feature with missing values as a function of other features, and uses that estimate for imputation
- At each step, a feature column is designated as output **y**, and the other feature columns are treated as inputs **X**

In [ ]:
from sklearn.experimental import enable_iterative_imputer  
from sklearn.impute import IterativeImputer

imp = IterativeImputer(random_state=42) # can specify the estimator - default is BayesianRidge

imputed_df = df.fit_transform(df.values)
imputed_df.isnull().any().any()


# Conclusion


**THERE ISN'T A PERFECT SOLUTION** 

*It really depends on the dataset and why is the value missing*